In [33]:
import pandas as pd
import numpy as np

In [34]:
df = pd.read_excel("C:\\Users\\Utsav\\Desktop\\ITR\\Mama\\AY_2025_26\\Mama_Capital_Gain.xlsx")
df.head()

,Ticker,BuyDate,BuyAmt,SellDate,Sell Amt,STCG,LTCG,Total,GST,Brokerage,Misc,STT_CTT,TotalBrokerage,NetPnL
0,AAVAS Financiers Ltd,2024-01-01,10946.90,2024-07-19,12246.14,1299.24,0.0,1299.24,6.43,35.72,2.26,23.18,67.59,1276.06
1,AAVAS Financiers Ltd,2024-01-02,11044.21,2024-07-19,12246.14,1201.93,0.0,1201.93,6.43,35.72,2.28,23.28,67.71,1178.64
2,AAVAS Financiers Ltd,2024-02-06,4317.10,2024-07-19,5248.35,931.24,0.0,931.24,4.92,27.31,0.88,9.55,42.65,921.70
3,Asian Paints Ltd,2024-01-23,9260.38,2024-07-15,8882.05,-378.33,0.0,-378.33,5.70,31.67,1.81,18.13,57.31,-396.46
4,Asian Paints Ltd,2024-01-31,8896.40,2024-07-15,8882.05,-14.35,0.0,-14.35,5.70,31.67,1.73,17.77,56.86,-32.12


In [35]:
# Add 'soldAt' column: 'B' if SellDate < 2024-07-23, else 'A'
df['soldAt'] = df['SellDate'].apply(lambda x: 'B' if pd.to_datetime(x) < pd.to_datetime('2024-07-23') else 'A')
df.head()

,Ticker,BuyDate,BuyAmt,SellDate,Sell Amt,STCG,LTCG,Total,GST,Brokerage,Misc,STT_CTT,TotalBrokerage,NetPnL,soldAt
0,AAVAS Financiers Ltd,2024-01-01,10946.90,2024-07-19,12246.14,1299.24,0.0,1299.24,6.43,35.72,2.26,23.18,67.59,1276.06,B
1,AAVAS Financiers Ltd,2024-01-02,11044.21,2024-07-19,12246.14,1201.93,0.0,1201.93,6.43,35.72,2.28,23.28,67.71,1178.64,B
2,AAVAS Financiers Ltd,2024-02-06,4317.10,2024-07-19,5248.35,931.24,0.0,931.24,4.92,27.31,0.88,9.55,42.65,921.70,B
3,Asian Paints Ltd,2024-01-23,9260.38,2024-07-15,8882.05,-378.33,0.0,-378.33,5.70,31.67,1.81,18.13,57.31,-396.46,B
4,Asian Paints Ltd,2024-01-31,8896.40,2024-07-15,8882.05,-14.35,0.0,-14.35,5.70,31.67,1.73,17.77,56.86,-32.12,B


In [36]:
# Define bins for SellDate
bins = [pd.Timestamp('1900-01-01'), pd.Timestamp('2024-06-15'), pd.Timestamp('2024-09-15'),
    pd.Timestamp('2024-12-15'), pd.Timestamp('2025-03-15'), pd.Timestamp('2025-03-31')]
labels = ['Upto 15/6', '16/6 to 15/9', '16/9 to 15/12', '16/12 to 15/3', '16/3 to 31/3']

df['SellDateBin'] = pd.cut(df['SellDate'], bins=bins, labels=labels, right=True)

# Calculation function
def calc_pnl(subset):
    return (subset['Sell Amt'] - (subset['BuyAmt'] + subset['TotalBrokerage'])).groupby(subset['SellDateBin']).sum()

# Prepare result dictionary
result = {}

# STCG != 0 & soldAt='B'
result['STCG != 0 & soldAt=B'] = calc_pnl(df[(df['STCG'] != 0) & (df['soldAt'] == 'B')])

# STCG != 0 & soldAt='A'
result['STCG != 0 & soldAt=A'] = calc_pnl(df[(df['STCG'] != 0) & (df['soldAt'] == 'A')])

# LTCG != 0 & soldAt='B'
result['LTCG != 0 & soldAt=B'] = calc_pnl(df[(df['LTCG'] != 0) & (df['soldAt'] == 'B')])

# LTCG != 0 & soldAt='A'
result['LTCG != 0 & soldAt=A'] = calc_pnl(df[(df['LTCG'] != 0) & (df['soldAt'] == 'A')])

# Combine into DataFrame
pnl_df = pd.DataFrame(result).T[labels].fillna(0)
pnl_df.columns = labels
pnl_df

C:\Users\Utsav\AppData\Local\Temp\ipykernel_16852\1700824723.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return (subset['Sell Amt'] - (subset['BuyAmt'] + subset['TotalBrokerage'])).groupby(subset['SellDateBin']).sum()


,Upto 15/6,16/6 to 15/9,16/9 to 15/12,16/12 to 15/3,16/3 to 31/3
STCG != 0 & soldAt=B,-33321.43,-3569.53,0.00,0.00,0.0
STCG != 0 & soldAt=A,0.00,24510.64,0.00,0.00,0.0
LTCG != 0 & soldAt=B,145183.57,0.00,0.00,0.00,0.0
LTCG != 0 & soldAt=A,0.00,104577.52,532203.32,105208.46,0.0


In [37]:
# STCG != 0 and soldAt == 'B'
stcg_b = df[(df['STCG'] != 0) & (df['soldAt'] == 'B')]
sum_buyamt_stcg_b = stcg_b['BuyAmt'].sum()
sum_sellamt_stcg_b = stcg_b['Sell Amt'].sum()
sum_brokerage_stcg_b = stcg_b['TotalBrokerage'].sum()

# STCG != 0 and soldAt == 'A'
stcg_a = df[(df['STCG'] != 0) & (df['soldAt'] == 'A')]
sum_buyamt_stcg_a = stcg_a['BuyAmt'].sum()
sum_sellamt_stcg_a = stcg_a['Sell Amt'].sum()
sum_brokerage_stcg_a = stcg_a['TotalBrokerage'].sum()

# LTCG != 0 and soldAt == 'B'
ltcg_b = df[(df['LTCG'] != 0) & (df['soldAt'] == 'B')]
sum_buyamt_ltcg_b = ltcg_b['BuyAmt'].sum()
sum_sellamt_ltcg_b = ltcg_b['Sell Amt'].sum()
sum_brokerage_ltcg_b = ltcg_b['TotalBrokerage'].sum()

# LTCG != 0 and soldAt == 'A'
ltcg_a = df[(df['LTCG'] != 0) & (df['soldAt'] == 'A')]
sum_buyamt_ltcg_a = ltcg_a['BuyAmt'].sum()
sum_sellamt_ltcg_a = ltcg_a['Sell Amt'].sum()
sum_brokerage_ltcg_a = ltcg_a['TotalBrokerage'].sum()

# Create a DataFrame for the summary output
summary_output = pd.DataFrame({
    'Type': [
        "STCG != 0 & soldAt='B'",
        "STCG != 0 & soldAt='A'",
        "LTCG != 0 & soldAt='B'",
        "LTCG != 0 & soldAt='A'"
    ],
    'BuyAmt': [
        sum_buyamt_stcg_b,
        sum_buyamt_stcg_a,
        sum_buyamt_ltcg_b,
        sum_buyamt_ltcg_a
    ],
    'SellAmt': [
        sum_sellamt_stcg_b,
        sum_sellamt_stcg_a,
        sum_sellamt_ltcg_b,
        sum_sellamt_ltcg_a
    ],
    'TotalBrokerage': [
        sum_brokerage_stcg_b,
        sum_brokerage_stcg_a,
        sum_brokerage_ltcg_b,
        sum_brokerage_ltcg_a
    ]
})
summary_output

,Type,BuyAmt,SellAmt,TotalBrokerage
0,STCG != 0 & soldAt='B',929258.45,897866.37,5498.88
1,STCG != 0 & soldAt='A',287582.26,313828.37,1735.47
2,LTCG != 0 & soldAt='B',189727.53,336395.93,1484.83
3,LTCG != 0 & soldAt='A',465762.47,1212896.58,5144.81


In [38]:
# Sum BuyAmt and SellAmt for soldAt='B' and soldAt='A' where STCG != 0
sum_buyamt_stcg = df.loc[(df['soldAt'] == 'B') & (df['STCG'] != 0), 'BuyAmt'].sum()
sum_sellamt_stcg = df.loc[(df['soldAt'] == 'B') & (df['STCG'] != 0), 'Sell Amt'].sum()

sum_buyamt_stcg_A = df.loc[(df['soldAt'] == 'A') & (df['STCG'] != 0), 'BuyAmt'].sum()
sum_sellamt_stcg_A = df.loc[(df['soldAt'] == 'A') & (df['STCG'] != 0), 'Sell Amt'].sum()

# Sum BuyAmt and SellAmt for soldAt='B' and soldAt='A' where LTCG != 0
sum_buyamt_ltcg = df.loc[(df['soldAt'] == 'B') & (df['LTCG'] != 0), 'BuyAmt'].sum()
sum_sellamt_ltcg = df.loc[(df['soldAt'] == 'B') & (df['LTCG'] != 0), 'Sell Amt'].sum()

sum_buyamt_ltcg_A = df.loc[(df['soldAt'] == 'A') & (df['LTCG'] != 0), 'BuyAmt'].sum()
sum_sellamt_ltcg_A = df.loc[(df['soldAt'] == 'A') & (df['LTCG'] != 0), 'Sell Amt'].sum()

print("STCG != 0, soldAt='B': BuyAmt =", sum_buyamt_stcg, ", SellAmt =", sum_sellamt_stcg)
print("STCG != 0, soldAt='A': BuyAmt =", sum_buyamt_stcg_A, ", SellAmt =", sum_sellamt_stcg_A)
print("LTCG != 0, soldAt='B': BuyAmt =", sum_buyamt_ltcg, ", SellAmt =", sum_sellamt_ltcg)
print("LTCG != 0, soldAt='A': BuyAmt =", sum_buyamt_ltcg_A, ", SellAmt =", sum_sellamt_ltcg_A)

STCG != 0, soldAt='B': BuyAmt = 929258.4500000002 , SellAmt = 897866.37
STCG != 0, soldAt='A': BuyAmt = 287582.26 , SellAmt = 313828.37000000005
LTCG != 0, soldAt='B': BuyAmt = 189727.52999999997 , SellAmt = 336395.93000000005
LTCG != 0, soldAt='A': BuyAmt = 465762.47000000003 , SellAmt = 1212896.58


In [39]:
pivot = df.pivot_table(
    index='soldAt',
    values=['STCG', 'LTCG', 'TotalBrokerage', 'BuyAmt', 'Sell Amt'],
    aggfunc='sum'
)
pivot.reset_index()

,soldAt,BuyAmt,LTCG,STCG,Sell Amt,TotalBrokerage
0,A,753344.73,747134.11,26246.10,1526724.95,6880.28
1,B,1118985.98,146668.37,-31392.01,1234262.30,6983.71
